In [ ]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()

### Sección 5: Ingeniería de Características (con Discretización)

#### Resumen Ejecutivo de la Sección

Esta sección transforma el `df_cleaned` en un formato listo para el modelado, con un enfoque en la **discretización** de variables continuas. Las transformaciones clave son:

1.  **Discretización mediante Codificación Ordinal:** Las variables continuas de medición (ej. `AS_TOT_mg/L`) se discretizaron utilizando las clasificaciones de calidad predefinidas del EDA (ej. `CALIDAD_AS`). Se estableció un mapa ordinal para convertir etiquetas como 'Potable - Excelente', 'Apta', y 'No Apta' en valores numéricos ordenados (0, 1, 2, etc.). Esto crea características ordinales que capturan los niveles de calidad definidos por el negocio.
2.  **One-Hot Encoding de `CONTAMINANTES`:** La columna de texto `CONTAMINANTES` se transformó en columnas binarias para cada tipo de contaminante, permitiendo al modelo interpretar cada uno como una característica independiente.
3.  **Eliminación de Columnas Redundantes y No Informativas:** Se eliminaron las columnas de alta cardinalidad (`MUNICIPIO`, `ACUIFERO`), los identificadores únicos (`CLAVE`, `SITIO`), la columna `CONTAMINANTES` original, y, de manera crucial, las **columnas de medición continua originales** (ej. `AS_TOT_mg/L`), ya que ahora están representadas por sus contrapartes ordinales (discretizadas).

El resultado es el DataFrame `df_featured`, un conjunto de datos numérico donde las mediciones continuas clave han sido reemplazadas por bins ordinales significativos.

In [ ]:
# --- 1. Discretización: Codificación Ordinal de las Columnas de Calidad ---

# Definir un mapa ordinal coherente para todas las etiquetas de calidad.
# El orden va de la mejor calidad (0) a la peor calidad.
ordinal_map = {
    # Calidad General y Metales
    'Potable - Excelente': 0, 'Buena calidad': 0, # Mejor calidad
    'Apta como FAAP': 1, 'Calidad aceptable': 1, # Calidad media
    'No apta como FAAP': 2, 'Mala calidad': 2,   # Peor calidad
    # Dureza
    'Suave': 0, 'Ligeramente dura': 1, 'Dura': 2, 'Muy dura': 3,
    # Salinidad
    'Agua dulce': 0, 'Agua ligeramente salina': 1, 'Agua salina': 2,
    # Riego (SDT_ra)
    'Baja': 0, 'Media': 1, 'Alta': 2, 'Muy Alta': 3,
    # Coliformes
    'Ausencia': 0, 'Presencia': 1
}

# Identificar las columnas de calidad categóricas
quality_cols = [col for col in df_featured.columns if col.startswith('CALIDAD_')]

# Aplicar el mapeo ordinal
for col in quality_cols:
    df_featured[col] = df_featured[col].map(ordinal_map)
print(f"Se han codificado ordinalmente {len(quality_cols)} columnas de calidad.")

# --- 2. One-Hot Encoding de la columna 'CONTAMINANTES' ---
contaminant_list = df_featured['CONTAMINANTES'].str.split(', ').explode().unique()
unique_contaminants = sorted([c for c in contaminant_list if c != 'Sin Contaminantes'])
for contaminant in unique_contaminants:
    new_col_name = f"CONTAMINANTE_{contaminant}"
    df_featured[new_col_name] = df_featured['CONTAMINANTES'].apply(lambda x: 1 if contaminant in x else 0)
print(f"Se han creado {len(unique_contaminants)} nuevas columnas para contaminantes.")

# --- 3. Eliminación de Columnas No Relevantes o Redundantes ---
cols_to_drop = [
    # Identificadores y alta cardinalidad
    'CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'PERIODO',
    # Columna original de contaminantes (redundante)
    'CONTAMINANTES',
    # Columnas de mediciones continuas (ahora representadas por bins ordinales)
    'ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L', 'FLUORUROS_mg/L', 'DUR_mg/L', 
    'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L', 'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 
    'HG_TOT_mg/L', 'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L'
]
df_featured.drop(columns=cols_to_drop, inplace=True)
print(f"Se eliminaron {len(cols_to_drop)} columnas no relevantes o redundantes.")

# --- 4. Codificación Binaria de Columnas de Cumplimiento y Semáforo ---
compliance_cols = [col for col in df_featured.columns if 'CUMPLE_CON' in col or 'SEMAFORO' in col]
binary_mapping = {'SI': 0, 'NO': 1, 'VERDE': 0, 'AMARILLO': 1, 'ROJO': 1}
for col in compliance_cols:
    df_featured[col] = df_featured[col].map(binary_mapping)
print("Se han codificado binariamente las columnas de cumplimiento y semáforo.")

# --- Verificación Final ---
print(f"\nForma final del DataFrame: {df_featured.shape}")
print("Vista previa del DataFrame transformado:")
display(df_featured.head())